In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [18]:
cap1 = cv2.VideoCapture('videos/queen1.mp4')
cap2 = cv2.VideoCapture('videos/queen2.mp4')

# cap2 프레임 스킵
skips = 492
for _ in range(skips):
    ret, frame = cap2.read()

pose1 = mp_pose.Pose()
pose2 = mp_pose.Pose()

while True:
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    if not (ret1 and ret2):
        break

    # mediapipe pose estimate
    results1 = pose1.process(cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB))
    results2 = pose2.process(cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB))

    # blank the frames
    height1, width1, _ = frame1.shape
    height2, width2, _ = frame2.shape
    frame1 = np.zeros((height1, width1, 3), dtype=np.uint8)
    frame2 = np.zeros((height2, width2, 3), dtype=np.uint8)

    # draw skeletons
    mp_drawing.draw_landmarks(
            frame1,
            results1.pose_landmarks,
            mp.solutions.pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    mp_drawing.draw_landmarks(
            frame2,
            results2.pose_landmarks,
            mp.solutions.pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

    if results1.pose_landmarks and results2.pose_landmarks:
        # make 2D array to 1D
        features1 = np.zeros(66)
        features2 = np.zeros(66)

        for i, landmark in enumerate(results1.pose_landmarks.landmark):
            features1[i*2] = landmark.x
            features1[i*2+1] = landmark.y

        for i, landmark in enumerate(results2.pose_landmarks.landmark):
            features2[i*2] = landmark.x
            features2[i*2+1] = landmark.y

        features1 = [features1]
        features2 = [features2]

        # calculate similarity between two arrays of pose estimation
        sim_score = cosine_similarity(features1, features2)[0][0]
        if sim_score > 0.98:
            cv2.putText(frame2, f"{sim_score}", (20, 700), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            cv2.putText(frame2, f"{sim_score}", (20, 700), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        cv2.putText(frame2, "No frame", (20, 700), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow('frame1', frame1)
    cv2.imshow('frame2', frame2)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap1.release()
cap2.release()
cv2.destroyAllWindows()